In [1]:
#import Dependencies
import pandas as pd
import numpy as np
import tweepy
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import re
import csv
from textblob import TextBlob

In [2]:
from config import (consumer_key,consumer_secret,access_token,access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [3]:
# Target Search Term
target_terms = ("@AlcoholDrugFdn","@drugsandmehub","@ondrugaddiction","@FindRecovery","@narconon","@Alcohoilic","@Drug_Topics","@FDATobacco")

In [4]:
# Counter
# Variables for holding sentiments
sentiments = []

# Loop through all target users
for target in target_terms:
    counter = 1

    for x in range(5):

    #Get tweets from homefeed
        public_tweets = api.user_timeline(target, page=x,count=200)


        for tweet in public_tweets:
            compound = analyzer.polarity_scores(tweet["text"])["compound"]
            pos = analyzer.polarity_scores(tweet["text"])["pos"]
            neu = analyzer.polarity_scores(tweet["text"])["neu"]
            neg = analyzer.polarity_scores(tweet["text"])["neg"]
            text=tweet['text']
            date=tweet['created_at']


 # Add sentiments for each tweet into a list
            sentiments.append({"Tweets":text,
                       "Positive": pos,
                       "Negative": neg})

# Add to counter 
            counter += 1


In [5]:
# Convert sentiments to DataFrame
sentiments_pd = pd.DataFrame.from_dict(sentiments)
sentiments_pd.shape


(7989, 3)

In [6]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [7]:
sentiments_pd['tidy_tweet'] = np.vectorize(remove_pattern)(sentiments_pd['Tweets'], "@[\w]*") 
sentiments_pd.shape

(7989, 4)

In [8]:
def f(row):
    if (row['Positive']>row['Negative']):
        val = "Positive"
        
    elif (row['Positive']<row['Negative']):
        val = "Negative"
    else:
        val = "Neutral"
    return val

In [9]:
sentiments_pd['Sentiment'] = sentiments_pd.apply(f, axis=1)


In [10]:
sentiments_pd['tidy_tweet'] = sentiments_pd['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")

In [11]:
sentiments_pd

,Negative,Positive,Tweets,tidy_tweet,Sentiment
0,0.000,0.178,Congratulations to our CEO @erinlalor who has ...,Congratulations to our CEO who has been appoi...,Positive
1,0.000,0.468,Great turnout in Moree yesterday! https://t.co...,Great turnout in Moree yesterday https t co...,Positive
2,0.131,0.066,RT @WSMLLEN: Shout out to all #Wimmera #Mallee...,RT Shout out to all #Wimmera #Mallee sports ...,Negative
3,0.000,0.000,You’ve probably heard about the deadly ‘monkey...,You ve probably heard about the deadly monkey...,Neutral
4,0.000,0.000,Text the effects is a drug information service...,Text the effects is a drug information service...,Neutral
5,0.000,0.118,Local to Moree? An initiative led by the Moree...,Local to Moree An initiative led by the Moree...,Positive
6,0.117,0.187,RT @sunriseon7: In a clear indication of Austr...,RT In a clear indication of Australia s grow...,Positive
7,0.000,0.120,RT @cracksintheice: How has the Cracks in the ...,RT How has the Cracks in the Ice website hel...,Positive
8,0.000,0.000,RT @pos_choices: One of the most common questi...,RT One of the most common questions parents ...,Neutral
9,0.000,0.086,RT @DrinkTankAU: On Drink Tank Summer Series -...,RT On Drink Tank Summer Series highlightin...,Positive


In [13]:
new_df = sentiments_pd[sentiments_pd.Sentiment != 'Neutral']

In [14]:
drug_text=new_df.drop(['Positive','Negative','Tweets'],axis=1)


In [15]:
drug_text.shape

(5415, 2)

In [16]:
drug_text.to_csv("drug_text.txt", encoding="utf-8", sep=',',index=False)

In [18]:
drug_text.to_csv("drug_text.csv", encoding="utf-8", sep=',',index=False)

In [2]:
!jupyter nbconvert --to script Sentiment_Analysis-alcohol_drugs.ipynb

[NbConvertApp] Converting notebook Sentiment_Analysis-alcohol_drugs.ipynb to script
[NbConvertApp] Writing 2851 bytes to Sentiment_Analysis-alcohol_drugs.py
